In [2]:
%matplotlib inline
import pymc3 as pm
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import pymc3 as pm
import theano.tensor as tt
from theano.tensor import _shared
import zhusuan as zs

In [82]:
example_data = np.array([1000, 869, 743, 653, 593, 551, 517, 491])
def n_lost(data):
    lost = [None]
    for i in range(1, len(data)):
        lost.append(data[i - 1] - data[i])
    return lost

example_data_n_lost = n_lost(example_data)
n = len(example_data)
data = (example_data, example_data_n_lost)
data = np.asarray(data)
data

array([[1000, 869, 743, 653, 593, 551, 517, 491],
       [None, 131, 126, 90, 60, 42, 34, 26]], dtype=object)

In [83]:
alpha = 0.6680763
beta = 3.806001526

In [105]:
def P_T_is_t(alpha=alpha, beta=beta, num_periods=n):
    p = np.array([None, alpha / (alpha + beta)])
    for t in range(2, num_periods):
        pt = ((beta + t - 2) / (alpha + beta + t - 1)) * p[t-1]
        p = np.append(p,pt)
    return p
    
def survival_function(P_T_is_t=P_T_is_t, num_periods=n):
    p = P_T_is_t()
    s = np.array([None, 1 - p[1]])
    for t in range(2, num_periods):
        s = np.append(s, s[t-1] - p[t])
    return s
    
def logp(value):
    active = value[0,:]
    lost = value[1,:]

    # Those who've churned along the way...
    p = P_T_is_t()
    died = np.multiply(np.log(np.array(p[1:], dtype=np.float64)), lost[1:]) 

    # and those still active in last period
    sf = survival_function()
    still_active = np.log(sf[-1]) * active[-1]              
    return sum(died) + still_active

In [106]:
p = np.array(P_T_is_t())

In [107]:
logp(data)

-1611.1581473982744

In [104]:
np.log(np.array(p[1:], dtype=np.float64))

array([-1.90165315, -2.26509781, -2.56303853, -2.81558714, -3.03479418,
       -3.22846073, -3.40193068])